<a href="https://colab.research.google.com/github/fbeilstein/algorithms/blob/master/hashmaps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* **hash table** maps keys to values (dictionary in Python, std::unordered_map in C++)
* keys should be hashable
* **hash function** maps int argument to fixed range of integers

In [ ]:
# Find first duplicate
# Suppose that there can only be numbers 1 to 10

input = [1,2,5,3,5,3,3,4,6,3,3,2,2,4,6,6]

arr_contains = [False] * 10
for i in input:
  if arr_contains[i - 1]:
    print(i)
    break
  arr_contains[i-1] = True

5


In [ ]:
# Find first duplicate
# No information on range !

input = [1,2,5,3,5,3,3,4,6,3,3,2,2,4,6,6]

arr_contains = {} # hash map to the resque
for i in input:
  if i in arr_contains:
    print(i)
    break
  arr_contains[i] = True

5


In [ ]:
# Simplest hashing

def hash(x):
  return x % 10

hash(27), hash(33), hash(147)

(7, 3, 7)

$$
h(x) \neq h(y) \rightarrow x \neq y
$$
contrapositive:
$$
x = y \rightarrow h(x) = h(y)
$$
but equality of hashes does not imply anything

In [ ]:
# Different objects can be hashable

class Student:
  def __init__(self, name, age, mark):
    self.name = name
    self.age = age
    self.mark = mark

  # Bad hash, but still hash
  def hash(self):
    return (len(self.name) + 3*self.age + self.mark) % 10

x = Student('Ivanov Ivan', 20, 4)
x.hash()

5

* do not confuse hash for hash maps with cryptographic hash -- they have different purposes thus different requirements
* you may be concerned how quick your hash works

In [ ]:
# Hash function should be deterministic !
# Example below not allowed

counter = 0
def hash(x):
  global counter
  counter += 1
  return (x + counter)

hash(3), hash(3), hash(3)

(4, 5, 6)

hash function:
* deterministic
* uniform
* keys are hashable = keys are immutable

In [ ]:
# unhashable key ! ERROR
{[1,2]: 3}

TypeError: ignored

What about **collisions** (i.e. $h(x) = h(y)$ but $x \neq y$)?

There are many techniques, but 2 most popular:

* separate chaining (maintains data structure to hold all values in one bucket, e.g. list, binary tree, self-balancing tree, etc)
* open adressing (finds another place in hash table by offsetting)

Given a good hash-function and table size not much smaller than the number of elements:

operation|average|worst
---|---|---
insertion|$O(1)$|$$O(n)$$
removal|$O(1)$|$O(n)$
search|$O(1)$|$O(n)$

In [ ]:
#@title Visualization functions

!pip install lolviz
from IPython.display import clear_output
clear_output()

str_style_bigtbl = '''
<style>
.bigtable {
  border-collapse: collapse;
}

.bigtd {
  border: 3px solid #ffd4d3ff;
  min-width:30px;
  height: 30px;
  position: relative; 
  text-align:center; 
  color: #474747;
  font-size:20px;
  font-weight: bolder;
  padding: 19px;
}

</style>
'''

str_slide_html = '''
<div style="position:absolute; top:30px; left:900px;">

<div id="arrow_left" style="border-width: 1px; border-style:solid; float:left; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect x="38.564" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m45.796 34.331h-5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.068 34.267 2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m40.069 34.396 2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

<div id="arrow_right" style="border-width: 1px; border-style:solid; float:right; height:32px;">
<svg width="8.7464mm" height="8.7464mm" version="1.1" viewBox="0 0 8.7464 8.7464" xmlns="http://www.w3.org/2000/svg" xmlns:cc="http://creativecommons.org/ns#" xmlns:dc="http://purl.org/dc/elements/1.1/" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
<metadata>
<rdf:RDF>
<cc:Work rdf:about="">
<dc:format>image/svg+xml</dc:format>
<dc:type rdf:resource="http://purl.org/dc/dcmitype/StillImage"/>
<dc:title/>
</cc:Work>
</rdf:RDF>
</metadata>
<g transform="translate(-38.564 -29.908)">
<rect transform="scale(-1,1)" x="-47.31" y="29.908" width="8.7464" height="8.7464" fill-opacity=".16425" opacity=".97" stroke-miterlimit="10.433" stroke-width="0"/>
<path d="m40.078 34.331h5.8931" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.806 34.267-2.7141-2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
<path d="m45.805 34.396-2.7141 2.4238" fill="none" stroke="#fff" stroke-opacity=".98068" stroke-width=".5"/>
</g>
</svg>
</div>

</div>

<style>
#arrow_left:hover{color:blue;}
#arrow_right:hover{color:blue;}
</style>

<script  type="text/javascript">
//var slides = ["slide_1", "slide_2", "slide_3"];

var container = document.getElementById("all_slides");
var menu_conn = document.createElement("div");
menu_conn.style.height = "90px";
menu_conn.id = "menu_div";
container.prepend(menu_conn);

// crutches
var slides = [];
var tmp = container.getElementsByTagName("div");
for (var i = 0; i < tmp.length; ++i)
  if (!tmp[i].id || tmp[i].id != "menu_div")
  {
    tmp[i].classList.add('slide_style');
    slides.push(tmp[i]);
  }


var left_btn_id = "arrow_left";
var right_btn_id = "arrow_right";
var current_slide = 0;
var menu = [];

function change_slide_to(new_slide_idx)
{
  menu[current_slide].style.borderWidth = "0px";
  var old_slide = slides[current_slide]; //document.getElementById(slides[current_slide]);
  current_slide = new_slide_idx;
  menu[current_slide].style.borderWidth = "2px";
  var new_slide = slides[current_slide]; //document.getElementById(slides[current_slide]);
  old_slide.style.display='none';
  new_slide.style.display='block';
}

function slide_left()
{
  var new_idx = current_slide - 1;
  if (new_idx < 0) {new_idx = 0;}
  change_slide_to(new_idx);
}

function slide_right()
{
  var new_idx = current_slide + 1;
  if (new_idx >= slides.length) {new_idx = slides.length - 1;}
  change_slide_to(new_idx);
}

function change(obj, is_correct)
{
//   alert(obj.parentNode.rowIndex);
    var rowId = obj.parentNode.rowIndex;
    var table = obj.parentNode.parentNode.parentNode;
    var rowsNotSelected = table.getElementsByTagName('tr');
    for (var row = 0; row < rowsNotSelected.length; row++)
    {
        rowsNotSelected[row].cells[0].style.backgroundColor = "white";
    }
    var rowSelected = table.getElementsByTagName('tr')[rowId];
    if (is_correct > 0)
    {
        rowSelected.cells[0].style.backgroundColor = "#66bb6aa9";
        menu[current_slide].style.backgroundColor = "#66bb6aa9";
    }
    else
    {
        rowSelected.cells[0].style.backgroundColor = "#ff3a3981";
        menu[current_slide].style.backgroundColor = "#ff3a3981";
    }
}

function make_callback(obj, is_correct)
{
  return function() { change(obj, is_correct); }
}

function change_callback(new_idx)
{
  return function() { change_slide_to(new_idx); }
}

document.getElementById(left_btn_id).addEventListener("click", slide_left);
document.getElementById(right_btn_id).addEventListener("click", slide_right);
slides[current_slide].style.display='block'; //document.getElementById(slides[current_slide]).style.display='block';

// create upper menu
upper_menu = document.getElementById("menu_div");

var tbl      = document.createElement("table");
var tbl_body = document.createElement("tbody");
var tbl_row  = document.createElement("tr");
for (idx = 0; idx < slides.length; idx++)
{
    var tbl_cell = document.createElement("td");
    var cell_text = document.createTextNode(idx+1);
    tbl_cell.appendChild(cell_text);
    tbl_cell.onclick = change_callback(idx);
    tbl_row.appendChild(tbl_cell);
    menu.push(tbl_cell);
    tbl_cell.style.borderWidth = "0px";

    var slide = slides[idx]; //document.getElementById(slides[idx]);
    var all_tables = slide.getElementsByTagName("table");
    for (t_i = 0; t_i < all_tables.length; t_i++)
    {
      var table = all_tables[t_i];
      if (!table.classList.contains("question_style"))
        continue;
      var rows = table.getElementsByTagName('tr');
      for (r_i = 0; r_i < rows.length; r_i++)
      {
        var question_cell = rows[r_i].cells[0];
        question_cell.classList.add("highlight");
        if (question_cell.classList.contains("ok"))
        {
          question_cell.onclick = make_callback(question_cell, 1);
        } else {
          question_cell.onclick = make_callback(question_cell, 0);
        }
        

        var cell = rows[r_i].insertCell(0);
        cell.width = "50px";
      }
    }
}
tbl_body.appendChild(tbl_row);
tbl.appendChild(tbl_body);
upper_menu.appendChild(tbl);
tbl.classList.add('menu_style');
menu[current_slide].style.borderWidth = "2px";

</script>

<style>
    .slide_style {
      max-width:750px;
      display:none;
      font: 14pt/18pt sans-serif;
      position: static;
      top: 80px;
      left: 0px;
    }

    .question_style {
        width:600px;
        font: 16pt/14pt sans-serif bold;
        text-align:left;
        cursor: default;
    }
    .question_style td{ 
        padding:7px;
        height: 50px;
        border:#4e95f4 0px solid;
        text-align:left;
        font: 13pt/16pt sans-serif;
    }
    .highlight:hover{
        background-color: #00000019
    }
    .menu_style {
        height:50px;
        font: 16pt/14pt sans-serif bold;
        text-align:left;
        cursor: default;
    }    
    .menu_style td{ 
        width:50px;
        padding:7px;
        border:#787878ff solid;
        text-align:center;
        font: 16pt/14pt sans-serif;
        color: #606060ff;
    }
</style>
'''

all_slides = []

def publish_slides():
  result = '<div id="all_slides">'
  global all_slides
  for e in all_slides:
    result += '<div>\n' + e + '\n</div>\n'
  result += '</div>\n'
  all_slides = []
  import IPython
  from google.colab import output
  display(IPython.display.HTML(str_style_bigtbl + str_slide_html + result))

def append_slide(dict_vars):
  from lolviz import objviz
  import graphviz
  g = objviz(dict_vars)
  src = graphviz.Source(g.source)
  global all_slides
  all_slides += [src.pipe(format='svg').decode("utf-8")]

In [ ]:
def get_hash(x, max_val=5):
  if type(x) == int:
    return x % max_val
  if type(x) == str:
    hash = 0
    base = 17
    for ch in x:
      hash = (hash*base + ord(ch)) % max_val
    return hash


class List:
  class Node:
    def __init__(self, key, value, next=None):
      self.key   = key
      self.value = value
      self.next  = next

  def __init__(self):
    self.head = None

  def push_front(self, key, val):
    self.head = List.Node(key, val, self.head)
  
  def del_by_key(self, key):
    p = List.Node(None, None, self.head)
    while p.next and p.next.key != key: p = p.next;
    if not p.next: return;
    if self.head == p.next: self.head = p.next.next;
    p.next = p.next.next

  def search_by_key(self, key):
    p = self.head
    while p and p.key != key: p = p.next;
    if p: return p.value
    return None

class HashTable:
  def __init__(self, size=5):
    self._data = [List() for _ in range(size)]
    self._size = size

  def add(self, key, value):
    idx = get_hash(key, self._size)
    self._data[idx].push_front(key, value) # we need key as well  

  def remove(self, key):
    idx = get_hash(key, self._size)
    self._data[idx].del_by_key(key)

  def get_value(self, key):
    idx = get_hash(key, self._size)
    return self._data[idx].search_by_key(key)


htable = HashTable()

htable.add(12, 4)
append_slide({"HT": htable})
htable.add(49, 7)
append_slide({"HT": htable})
htable.add(37, 17)
append_slide({"HT": htable})
htable.remove(1)
append_slide({"HT": htable})
htable.remove(37)
append_slide({"HT": htable})
val = htable.get_value(49)
append_slide({"HT": htable, "val": val})


publish_slides()

In [ ]:
def get_hash(x, max_val=5):
  if type(x) == int:
    return x % max_val
  if type(x) == str:
    hash = 0
    base = 17
    for ch in x:
      hash = (hash*base + ord(ch)) % max_val
    return hash

def get_another_hash(x, max_val=5):
  if type(x) == int:
    return x**2 % max_val
  if type(x) == str:
    hash = 0
    base = 37
    for ch in x:
      hash = (hash*base + ord(ch)) % max_val
    return hash
    
class HashTable:
  def __init__(self, collision_resolver, size=5):
    self._data = [None] * size
    self._size = size
    self._resolve = collision_resolver

  def add(self, key, value):
    idx = get_hash(key, self._size)
    iteration = 0
    while self._data[idx] and self._data[idx][0] != key and iteration < self._size: 
      idx = self._resolve(key, idx, iteration) % self._size
      iteration += 1
    if iteration >= self._size: 
      raise Exception("out of space")
    self._data[idx] = (key, value) # we need key as well  

  def remove(self, key):
    idx = get_hash(key, self._size)
    iteration = 0
    while self._data[idx] and self._data[idx][0] != key and iteration < self._size:
      idx = self._resolve(key, idx, iteration) % self._size
      iteration += 1
    if self._data[idx] and self._data[idx][0] == key:
      self._data[idx] = None

  def get_value(self, key):
    idx = get_hash(key, self._size)
    iteration = 0
    while self._data[idx] and self._data[idx][0] != key and iteration < self._size:
      idx = self._resolve(key, idx, iteration) % self._size
      iteration += 1
    if self._data[idx] and self._data[idx][0] == key:
      return self._data[idx][1]
    return None


linear_probing = lambda key, hashed_key, iter: hashed_key + 3 * iter + 17 
quadratic_probing = lambda key, hashed_key, iter: hashed_key + 3 * iter**2 + iter + 17 
double_hashing = lambda key, hashed_key, iter: get_another_hash(key, 49)*iter
rnd = lambda key, hashed_key, iter: hashed_key**3 % 17 if iter == 0 else rnd(key, hashed_key, iter-1)

htable = HashTable(rnd, 5)

htable.add(12, 4)
append_slide({"HT": htable})
htable.add(49, 7)
append_slide({"HT": htable})
htable.add(37, 17)
append_slide({"HT": htable})
htable.remove(1)
append_slide({"HT": htable})
htable.remove(37)
append_slide({"HT": htable})
val = htable.get_value(49)
append_slide({"HT": htable, "val": val})


publish_slides()

In [ ]:
class MyHashMap {
public:
    MyHashMap() 
    {
    }
    
    int h1(int key)
    {
        return key % size(v);
    }
    
    int h(int key, int i)
    {
        return (h1(key) + i) % size(v);
    }

    
    void put(int key, int value) 
    {
        auto p = h(key, 0);
        for(int i{0}; v[p].first >= 0 and v[p].first != key; ++i, p = h(key, i));
        v[p] = {key, value};
    }
    
    int get(int key) 
    {
        auto p = h(key, 0);
        for(int i{0}; v[p].first != -2; ++i, p = h(key, i))
            if(v[p].first == key) return v[p].second;
        
        return -1;
    }
    
    void remove(int key) 
    {
        auto p = h(key, 0);
        for(int i{0}; v[p].first != -2; ++i, p = h(key, i))
            if(v[p].first == key) v[p].first = -1;
    }
    
    vector<pair<int, int>> v = vector<pair<int, int>>(20000, make_pair(-2, 0));
};

### Number of Good Pairs

In [ ]:
class Solution:
    def numIdenticalPairs(self, nums: List[int]) -> int:
        cache = {}
        for num in nums:
            if num in cache:
                cache[num] += 1
            else:
                cache[num] = 1
        sum = 0
        for i in cache:
            sum += cache[i]*(cache[i]-1)//2
        return sum
        

###Maximum Erasure Value

In [ ]:
class Solution:
    def maximumUniqueSubarray(self, nums: List[int]) -> int:
        tail, sum, max_sum = 0, 0, 0
        used = {}
        for head,n in enumerate(nums):
            if n in used:
                new_tail = used[n]
                while tail <= new_tail:
                    used.pop(nums[tail])
                    sum -= nums[tail]
                    tail += 1
            used[n] = head
            sum += n
            max_sum = max(sum, max_sum)
                
        return max_sum
                
        
        

###Apply Discount Every n Orders

In [ ]:
class Cashier:

    def __init__(self, n: int, discount: int, products: List[int], prices: List[int]):
        self._prices = {product: price for product,price in zip(products,prices)}
        self._n = n
        self._discount = discount
        self._current_customer = 0
        

    def getBill(self, product: List[int], amount: List[int]) -> float:
        self._current_customer += 1
        mult = 1.0 if self._current_customer % self._n else (100 - self._discount)/100.0
        return mult * sum([self._prices[p] * a for p,a in zip(product, amount)])
        


# Your Cashier object will be instantiated and called as such:
# obj = Cashier(n, discount, products, prices)
# param_1 = obj.getBill(product,amount)

###Longest Arithmetic Subsequence of Given Difference

In [ ]:
class Solution:
    def longestSubsequence(self, arr: List[int], difference: int) -> int:
        cache = {}
        out = 0
        for el in arr:
            cache[el] = cache.get(el - difference, 0) + 1
            out = max(out, cache[el])
        return out
        